In [1]:
from pymongo import MongoClient
client = MongoClient(readPreference='secondary')
db = client.spotify_db
tracks_db = db.tracks_db
playlists_db = db.playlists_db
artists_db = db.artists_db
from pandas.io.json import json_normalize
import pandas as pd
from tqdm import tqdm
from collections import defaultdict
from scipy.sparse import csr_matrix
import numpy as np

def construct_model_data():
    datapoints = list(artists_db.find({}, {'genres': 1}))
    df = json_normalize(datapoints)
    df.set_index('_id', inplace=True)
    return df


raw_df = construct_model_data()
raw_df = raw_df[raw_df['genres'].map(lambda d: type(d) != float)]
df = raw_df[raw_df['genres'].map(lambda d: len(d)) > 0]


import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel



def get_genre_tuples(genres):
    tups = []
    if genres:
        if type(genres) == float:
            pass
        else:
            for genre in genres:
                genre_index = word2id[genre]
                tups.append((genre_index, 1))
    return tups

id2word = corpora.Dictionary(df.genres)
word2id = {v: k for k, v in id2word.items()}
corpus = [id2word.doc2bow(genres) for genres in df.genres]
corpus
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           passes=5,
                                           alpha='auto',
                                           per_word_topics=True)


import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

/Users/jeremy/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


In [2]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
6      0.165581  0.275833       1        1  7.313740
8      0.018105 -0.110937       2        1  6.410132
16    -0.280463  0.115343       3        1  6.117844
19    -0.088069  0.149351       4        1  5.984931
0      0.065737  0.094267       5        1  5.778894
9      0.047112 -0.028307       6        1  5.392431
2      0.099014 -0.070174       7        1  5.214723
10     0.131048  0.025731       8        1  5.139497
7     -0.002700 -0.046886       9        1  5.008343
3     -0.002073 -0.022074      10        1  4.980609
1     -0.272404  0.063331      11        1  4.890796
5      0.022363  0.048364      12        1  4.764633
11    -0.014878 -0.170981      13        1  4.763700
14     0.002858 -0.059171      14        1  4.525418
18     0.014642 -0.036890      15        1  4.437129
17    -0.005810 -0.186176      16        1  4.192976
12     0.118910  0.096962      17        1  4.191606
13    -0.036770 -0.057581      18        1  3.888714
4      0.027106 -0.020851      19        1  3.605502
15    -0.009310 -0.059158      20        1  3.398383, topic_info=     Category        Freq                     Term       Total  loglift  \
1012  Default  857.000000         background music  857.000000  30.0000   
640   Default  872.000000           electric blues  872.000000  29.0000   
778   Default  663.000000                    blues  663.000000  28.0000   
230   Default  558.000000             modern blues  558.000000  27.0000   
712   Default  581.000000                    focus  581.000000  26.0000   
...       ...         ...                      ...         ...      ...   
10    Topic20   71.136322  modern alternative rock   73.261429   3.3524   
730   Topic20   67.998634    brazilian composition   70.123741   3.3511   
205   Topic20   67.429031        minnesota hip hop   69.554138   3.3508   
932   Topic20   56.544338        canadian pop punk   58.669441   3.3450   
738   Topic20   53.477989          latin classical   55.603092   3.3429   

      logprob  
1012  30.0000  
640   29.0000  
778   28.0000  
230   27.0000  
712   26.0000  
...       ...  
10    -4.1847  
730   -4.2298  
205   -4.2382  
932   -4.4143  
738   -4.4700  

[677 rows x 6 columns], token_table=      Topic      Freq                    Term
term                                         
875       9  0.979395  21st century classical
375       2  0.981926              a cappella
790       9  0.991004        abstract hip hop
963      17  0.980424              acid house
487       4  0.994597               acid jazz
...     ...       ...                     ...
531       7  0.982049                   wonky
601       4  0.989570                   world
551       2  0.994067           world worship
549      14  0.989432                 worship
200       3  0.975810              yacht rock

[684 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 9, 17, 20, 1, 10, 3, 11, 8, 4, 2, 6, 12, 15, 19, 18, 13, 14, 5, 16])

In [6]:


from gensim.test.utils import datapath
# Save model to disk

lda_model.save(temp_file)
lda = LdaModel.load(temp_file)

<class 'str'>
<class 'str'>


AttributeError: 'str' object has no attribute 'save'

In [ ]:
def get_genre_tuples(genres):
    tups = []
    if genres:
        if type(genres) == float:
            pass
        else:
            for genre in genres:
                genre_index = word2id[genre]
                tups.append((genre_index, 1))
    return tups
lda_model[id2word.doc2bow([])]